In [2]:
import numpy as np
from pyscf import gto, scf, ao2mo
from pyscf.tools import molden, fcidump
from openfermion.ops import InteractionOperator
from openfermionpyscf import run_pyscf
import openfermion as of
import re
import glob
from pyscf import scf, gto, dft, mp, ao2mo, cc, mp, ci, fci
from pyscf.tools import fcidump
import json
from pyscf import gto,scf,mcscf, fci,lo
from pyscf.scf import ROHF, UHF, ROKS
import numpy as np
import pandas as pd
from pyscf.tools import fcidump
from pyscf.lib import chkfile

/Users/twwatts/.conda/envs/quantum/lib/python3.12/site-packages/pyscf/dft/libxc.py:771: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, corresponding to the original definition by Stephens et al. (issue 1480) and the same as the B3LYP functional in Gaussian. To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '


In [3]:
from openfermion.chem.molecular_data import spinorb_from_spatial

In [4]:
chkfiles = glob.glob("*_chkfile.chk")
chkfiles

['o_h_cc-pV5Z_chkfile.chk',
 'n2_h4_cc-pVTZ_chkfile.chk',
 'c_h3_cl_cc-pVTZ_chkfile.chk',
 'si2_cc-pVQZ_chkfile.chk',
 'c_s_cc-pV5Z_chkfile.chk',
 'p2_cc-pVTZ_chkfile.chk',
 's_cc-pV5Z_chkfile.chk',
 'h2_o_cc-pVTZ_chkfile.chk',
 'c_cc-pV5Z_chkfile.chk',
 'c2_h6_cc-pV5Z_chkfile.chk',
 'f2_cc-pVDZ_chkfile.chk',
 'li2_cc-pVDZ_chkfile.chk',
 'na_cc-pVQZ_chkfile.chk',
 'c_h2_singlet_cc-pVTZ_chkfile.chk',
 'p_h2_cc-pV5Z_chkfile.chk',
 'c_h_cc-pVDZ_chkfile.chk',
 'h_cl_cc-pVQZ_chkfile.chk',
 'f_cc-pVQZ_chkfile.chk',
 'na2_cc-pVTZ_chkfile.chk',
 'p_cc-pV5Z_chkfile.chk',
 'c2_h4_cc-pV5Z_chkfile.chk',
 's2_cc-pVQZ_chkfile.chk',
 'h_o_cl_cc-pVDZ_chkfile.chk',
 'cl_o_cc-pVQZ_chkfile.chk',
 'c_o2_cc-pV5Z_chkfile.chk',
 'c2_h2_cc-pVQZ_chkfile.chk',
 'p_h3_cc-pV5Z_chkfile.chk',
 'li_cc-pVTZ_chkfile.chk',
 'cl_f_cc-pVDZ_chkfile.chk',
 'c2_h2_cc-pV5Z_chkfile.chk',
 'n_cc-pVDZ_chkfile.chk',
 'c_o2_cc-pVQZ_chkfile.chk',
 'cl_o_cc-pV5Z_chkfile.chk',
 'p_h3_cc-pVQZ_chkfile.chk',
 'be_cc-pVTZ_chkfile.chk',


In [14]:
chk = 'li_cc-pVDZ_chkfile.chk'
mol = chkfile.load_mol(chk)
mf = scf.ROHF(mol)
scf_result_dic = chkfile.load(chk, 'scf')
mf.__dict__.update(scf_result_dic)
hcore = mf.get_hcore()

# Number of basis functions
eri = mol.intor('int2e')
eri_4d = ao2mo.restore(1, eri, mol.nao_nr())

ls, one_bodys, corr, x = of.circuits.low_rank_two_body_decomposition(
eri_4d,
truncation_threshold=1e-08,
final_rank=None,
spin_basis=False)

In [20]:
from scipy.linalg import logm

In [26]:
Rs=[]
JW_terms=[]
for one_body in one_bodys:
        # Diagonalize the one_body matrix
    eigenvalues, V = np.linalg.eigh(one_body)

    
    # Compute the matrix logarithm
    u_log = logm(V)
    Rs.append(u_log)

    hamiltonian = FermionOperator()
    for i, d in enumerate(eigenvalues):
        term = FermionOperator(((i, 1), (i, 0)), coefficient=d)
        hamiltonian += term

    jw_hamiltonian = jordan_wigner(hamiltonian)
    JW_terms.append(jw_hamiltonian)


In [29]:
np.round(Rs[0],3)

array([[ 0.   +0.j, -0.022+0.j,  0.356-0.j, -0.215+0.j,  0.73 +0.j,
         0.081-0.j, -0.449+0.j, -0.368-0.j,  0.262+0.j, -0.569+0.j,
        -0.289-0.j,  0.165+0.j, -0.135-0.j, -0.083+0.j,  0.43 -0.j,
         0.185-0.j, -0.403+0.j, -0.116-0.j, -0.936+0.j,  0.129+0.j,
        -0.041+0.j,  0.298+0.j, -0.145-0.j, -0.05 -0.j, -0.185-0.j,
        -0.281-0.j, -0.093-0.j,  0.22 +0.j],
       [ 0.022+0.j,  0.   -0.j, -0.051+0.j, -0.206-0.j,  0.045+0.j,
        -0.419-0.j,  0.239-0.j,  0.221-0.j,  0.679-0.j, -0.464-0.j,
         0.329-0.j,  0.116+0.j, -0.439-0.j,  0.541-0.j, -0.206+0.j,
        -0.724-0.j,  0.049-0.j, -0.42 +0.j,  0.104-0.j, -0.642+0.j,
        -0.289-0.j,  0.302-0.j,  0.071+0.j,  0.519-0.j, -0.035+0.j,
         0.181-0.j,  0.036-0.j, -0.347+0.j],
       [-0.356-0.j,  0.051-0.j, -0.   +0.j, -0.355+0.j,  0.087+0.j,
         0.282-0.j, -0.122+0.j, -0.542-0.j,  0.029-0.j, -0.238+0.j,
        -0.56 -0.j,  0.311-0.j,  0.06 +0.j,  0.259-0.j,  0.275-0.j,
         0.107-0.j, -0.604

In [16]:
import numpy as np
from openfermion.ops import FermionOperator
from openfermion.transforms import jordan_wigner
from openfermion.utils import hermitian_conjugated

# Define the number of qubits (orbitals)
num_qubits = 4

# Create a diagonal Hamiltonian
# H = sum_p d_p n_p, where d_p are eigenvalues and n_p = a^\dagger_p a_p
eigenvalues = [1, 2, 3, 4]
hamiltonian = FermionOperator()

for i, eigenvalue in enumerate(eigenvalues):
    term = FermionOperator(((i, 1), (i, 0)), coefficient=eigenvalue)
    hamiltonian += term

print("Fermionic Hamiltonian:")
print(hamiltonian)

# Perform Jordan-Wigner transformation
jw_hamiltonian = jordan_wigner(hamiltonian)

print("\nQubit Hamiltonian (after Jordan-Wigner transformation):")
print(jw_hamiltonian)


Fermionic Hamiltonian:
1.0 [0^ 0] +
2.0 [1^ 1] +
3.0 [2^ 2] +
4.0 [3^ 3]

Qubit Hamiltonian (after Jordan-Wigner transformation):
(5+0j) [] +
(-0.5+0j) [Z0] +
(-1+0j) [Z1] +
(-1.5+0j) [Z2] +
(-2+0j) [Z3]


In [7]:
from openfermion.ops import QubitOperator

def construct_hamiltonian(lambdas, f_coeffs):
    L = len(lambdas)  # Number of sum terms
    num_qubits = len(f_coeffs[0])  # Assuming f_coeffs is structured as [ [f_0^1, f_1^1, ..., f_N^1], ..., [f_0^L, ..., f_N^L] ]
    H = QubitOperator()

    for ell in range(L):
        lambda_ell = lambdas[ell]
        single_term = QubitOperator()

        # Construct the double summation term
        for p in range(num_qubits):
            for q in range(num_qubits):
                if p == q:
                    single_term += 0.25 * f_coeffs[ell][p] * f_coeffs[ell][q] * (QubitOperator() - QubitOperator(f'Z{p}'))
                else:
                    single_term += 0.25 * f_coeffs[ell][p] * f_coeffs[ell][q] * (QubitOperator() - QubitOperator(f'Z{p}')) * (QubitOperator() - QubitOperator(f'Z{q}'))

        # Assume R_ell and R_ell_dagger are identity, handle differently if not
        H += lambda_ell * single_term

    return H

# Example usage
lambdas = [1.0, 0.5]  # Example lambda values
f_coeffs = [[0.5, 0.3], [0.4, 0.1]]  # Example coefficients
H_jw = construct_hamiltonian(lambdas, f_coeffs)
print(H_jw)


-0.0825 [Z0] +
0.08499999999999999 [Z0 Z1] +
-0.02375 [Z1]
